In [1]:
from ptrail.core.Datasets import Datasets
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.utilities.conversions import Conversions
from ptrail.regularization.compression import Compression
import pandas as pd

In [2]:
dataset = PTRAILDataFrame(data_set=pd.read_csv('./data/compression_data.csv'),
                          datetime='DateTime', traj_id='traj_id', latitude='lat', longitude='lon')
dataset

trajectory      lat       lon   sog    cog  \
traj_id   DateTime                                                          
316002721 2020-04-06 05:42:48           0  37.6018 -122.6299   9.2  316.2   
          2020-04-06 05:45:48           0  37.6074 -122.6357   8.8  323.8   
          2020-04-06 05:47:17           0  37.6102 -122.6386   8.4  319.3   
          2020-04-06 05:50:18           0  37.6159 -122.6445   8.7  317.9   
          2020-04-06 05:51:19           0  37.6178 -122.6464   8.2  321.1   
...                                   ...      ...       ...   ...    ...   
368141480 2020-05-31 00:38:22          54  37.7700 -122.8892  10.1  333.4   
          2020-05-31 00:39:31          54  37.7728 -122.8913  10.6  323.7   
          2020-05-31 00:40:41          54  37.7755 -122.8939  10.3  323.2   
          2020-05-31 00:41:51          54  37.7785 -122.8960  10.3  331.5   
          2020-05-31 00:43:02          54  37.7813 -122.8983  10.4  324.3   

                               vessel_type                      flag  
traj_id   DateTime                                                    
316002721 2020-04-06 05:42:48           30                    Canada  
          2020-04-06 05:45:48           30                    Canada  
          2020-04-06 05:47:17           30                    Canada  
          2020-04-06 05:50:18           30                    Canada  
          2020-04-06 05:51:19           30                    Canada  
...                                    ...                       ...  
368141480 2020-05-31 00:38:22           30  United States of America  
          2020-05-31 00:39:31           30  United States of America  
          2020-05-31 00:40:41           30  United States of America  
          2020-05-31 00:41:51           30  United States of America  
          2020-05-31 00:43:02           30  United States of America  

[70738 rows x 7 columns]

In [3]:
# adapt from https://github.com/uestc-db/traj-compression/blob/master/batch/TD-TR/TD-TR.cpp

import numpy as np
import time

import pandas as pd


def calc_SED(pA, pI, pB):
    """
    It computes the Synchronous Euclidean Distance (SED) error

    :param pA: initial point
    :param pI: middle point
    :param pB: final point
    :return: SED error
    """
    pA_lat, pA_lon, pA_time = pA
    pI_lat, pI_lon, pI_time = pI
    pB_lat, pB_lon, pB_time = pB

    middle_dist = pI_time - pA_time
    total_dist = pB_time - pA_time
    if total_dist == 0:
        time_ratio = 0
    else:
        time_ratio = middle_dist / total_dist

    lat = pA_lat + (pB_lat - pA_lat) * time_ratio
    lon = pA_lon + (pB_lon - pA_lon) * time_ratio

    lat_diff = lat - pI_lat
    lon_diff = lon - pI_lon
    error = np.sqrt((lat_diff * lat_diff) + (lon_diff * lon_diff))
    return error


def calc_DP(pA, pI, pB):
    """
    It computes the Perpendicular Distance (PD)

    :param pA: initial point
    :param pI: middle point
    :param pB: final point
    :return: shortest distance
    """
    pA_lat, pA_lon, pA_time = pA
    pI_lat, pI_lon, pI_time = pI
    pB_lat, pB_lon, pB_time = pB

    # equation: (yA−yB)x−(xA−xB)y+xAyB−xByA=0.
    A = pA_lon - pB_lon
    B = - (pA_lat - pB_lat)
    C = pA_lat * pB_lon - pB_lat * pA_lon

    if A == 0 and B == 0:
        shortDist = 0
    else:
        shortDist = abs((A * pI_lat + B * pI_lon + C) / np.sqrt(A * A + B * B))

    return shortDist


def calc_AVS(pA, pI, pB):
    """
    It computes the absolute value of speed (AVS)

    :param pA: initial point
    :param pI: middle point
    :param pB: final point
    :return: AVS value
    """
    pA_lat, pA_lon, pA_time = pA
    pI_lat, pI_lon, pI_time = pI
    pB_lat, pB_lon, pB_time = pB

    d1 = np.sqrt((pI_lat - pA_lat) * (pI_lat - pA_lat) + (pI_lon - pA_lon) * (pI_lon - pA_lon))
    d2 = np.sqrt((pB_lat - pI_lat) * (pB_lat - pI_lat) + (pB_lon - pI_lon) * (pB_lon - pI_lon))

    v1 = 0
    v2 = 0
    if (pI_time - pA_time) > 0:
        v1 = d1 / (pI_time - pA_time)
    if (pB_time - pI_time) > 0:
        v2 = d2 / (pB_time - pI_time)
    AVS = abs(v2 - v1)

    return AVS


def calc_TR_SP(trajectory, dim_set, traj_time, epsilon, epsilon2, calc_func, calc_func2):
    """
    It compress the trajectory using the two compression techniques.
    It is a recursive method, dividing the trajectory and compression both parts.

    :param trajectory: a single trajectory or a part of if
    :param dim_set: the attributes in the dict trajectory
    :param traj_time: the array with the time in seconds of each point
    :param epsilon: the threshold for the first compression
    :param epsilon2: the threshold for second compression
    :param calc_func: the measure for the first selected compression
    :param calc_func2: the measure for the second selected compression
    :return: the compressed trajectory (dict)
    """
    new_trajectory = {}
    for dim in dim_set:
        new_trajectory[dim] = np.array([])
    traj_len = len(trajectory['lat'])

    # time in seconds
    dmax, idx, _ = traj_max_dists(trajectory, traj_time, calc_func)
    start_location = (trajectory['lat'][0], trajectory['lon'][0], traj_time[0])
    final_location = (trajectory['lat'][-1], trajectory['lon'][-1], traj_time[-1])
    middle_location = (trajectory['lat'][idx], trajectory['lon'][idx], traj_time[idx])
    d_idx = calc_func2(start_location, middle_location, final_location)

    trajectory['DateTime'] = trajectory['DateTime'].astype(str)

    if (dmax > epsilon) & (d_idx > epsilon2):
        traj1 = {}
        traj2 = {}
        for dim in dim_set:
            traj1[dim] = trajectory[dim][0:idx]
            traj2[dim] = trajectory[dim][idx:]

        # compression of the parts
        recResults1 = traj1
        if len(traj1['lat']) > 2:
            recResults1 = traj_compression(traj1, dim_set, traj_time[0:idx], calc_func, epsilon)

        recResults2 = traj2
        if len(traj2['lat']) > 2:
            recResults2 = traj_compression(traj2, dim_set, traj_time[idx:], calc_func, epsilon)

        for dim in dim_set:
            new_trajectory[dim] = np.append(new_trajectory[dim], recResults1[dim])
            new_trajectory[dim] = np.append(new_trajectory[dim], recResults2[dim])

    else:
        trajectory['DateTime'] = trajectory['DateTime'].astype(str)
        for dim in dim_set:
            new_trajectory[dim] = np.append(new_trajectory[dim], trajectory[dim][0])
            if traj_len > 1:
                new_trajectory[dim] = np.append(new_trajectory[dim], trajectory[dim][-1])

    return new_trajectory


def traj_max_dists(trajectory, traj_time, calc_func):
    """
    It computes the selected measure for all points in between.

    :param trajectory: a single dict trajectory having the keys as each attribute
    :param traj_time: an array with the seconds of each point
    :param calc_func: the measure for the selected compression
    :return: the maximum distance, the index that provide the maximum distance, and the average of distances
    """
    dmax = 0
    idx = 0
    ds = np.array([])
    traj_len = len(trajectory['lat'])
    # start and final points
    start_location = (trajectory['lat'][0], trajectory['lon'][0], traj_time[0])
    final_location = (trajectory['lat'][-1], trajectory['lon'][-1], traj_time[-1])
    for i in range(1, (traj_len - 1)):
        # middle point at index i
        middle = (trajectory['lat'][i], trajectory['lon'][i], traj_time[i])
        # compute the distance
        d = calc_func(start_location, middle, final_location)
        # get distances information
        ds = np.append(ds, d)
        if d > dmax:
            dmax = d
            idx = i

    return dmax, idx, ds.mean()


def traj_compression(trajectory, dim_set, traj_time, calc_func, epsilon):
    """
    It compress the trajectory using the compression technique.
    It is a recursive method, dividing the trajectory and compression both parts.

    :param trajectory: a single trajectory or a part of if
    :param dim_set: the attributes in the dict trajectory
    :param traj_time: the array with the time in seconds of each point
    :param calc_func: the measure for the selected compression
    :param epsilon: the threshold
    :return: the compressed trajectory (dict)
    """
    new_trajectory = {}
    for dim in dim_set:
        new_trajectory[dim] = np.array([])
    traj_len = len(trajectory['lat'])

    # time in seconds
    dmax, idx, _ = traj_max_dists(trajectory, traj_time, calc_func)
    trajectory['DateTime'] = trajectory['DateTime'].astype(str)

    # print(f'\tepsilon: {epsilon}, dmax: {dmax}, index: {idx}, trajlen: {traj_len}')
    if dmax > epsilon:
        traj1 = {}
        traj2 = {}
        for dim in dim_set:
            traj1[dim] = trajectory[dim][0:idx]
            traj2[dim] = trajectory[dim][idx:]

        # compression of the parts
        recResults1 = traj1
        if len(traj1['lat']) > 2:
            recResults1 = traj_compression(traj1, dim_set, traj_time[0:idx], calc_func, epsilon)

        recResults2 = traj2
        if len(traj2['lat']) > 2:
            recResults2 = traj_compression(traj2, dim_set, traj_time[idx:], calc_func, epsilon)

        for dim in dim_set:
            new_trajectory[dim] = np.append(new_trajectory[dim], recResults1[dim])
            new_trajectory[dim] = np.append(new_trajectory[dim], recResults2[dim])

    else:
        trajectory['DateTime'] = trajectory['DateTime'].astype(str)
        for dim in dim_set:
            new_trajectory[dim] = np.append(new_trajectory[dim], trajectory[dim][0])
            if traj_len > 1:
                new_trajectory[dim] = np.append(new_trajectory[dim], trajectory[dim][-1])

    return new_trajectory


def compression(dataset, metric='TR', verbose=True, alpha=1):
    """
    It compress the dataset of trajectories using the selected compression technique.

    :param dataset: dict dataset with trajectories
    :param metric: the compression technique or combination of them (Default: TR).
    :param verbose: if True, it shows the messages (Default: True).
    :param alpha: the predefined factor (Default: 1).
    :return: the compressed dataset, compression rate, and processing time.
    """
    # sys.setrecursionlimit(2200)
    metrics = {'TR': calc_SED,
               'DP': calc_DP,
               'SP': calc_AVS,
               'TR_SP': calc_TR_SP,
               'SP_TR': calc_TR_SP,
               'SP_DP': calc_TR_SP,
               'DP_SP': calc_TR_SP,
               'DP_TR': calc_TR_SP,
               'TR_DP': calc_TR_SP}

    calc_func = metrics[metric]

    mmsis = list(dataset.keys())
    new_dataset = {}
    compression_rate = np.array([])
    processing_time = np.array([])

    dim_set = dataset[mmsis[0]].keys()
    print("Dim set is: ", dim_set)

    if verbose:
        print(f"Compressing with {metric} and factor {alpha}")
    for id_mmsi in range(len(mmsis)):
        new_dataset[mmsis[id_mmsi]] = {}
        if verbose:
            print(f"\tCompressing {id_mmsi} of {len(mmsis)}")
        # trajectory a
        t0 = time.time()
        curr_traj = dataset[mmsis[id_mmsi]]
        # get time in seconds
        traj_time = curr_traj['DateTime'].astype('datetime64[s]')
        traj_time = np.hstack((0, np.diff(traj_time).cumsum().astype('float')))
        traj_time = traj_time / traj_time.max()
        # compress trajectory
        compress_traj = curr_traj
        try:
            if metric in ['TR_SP']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_SED)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_AVS)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_SED,
                                          calc_AVS)
            elif metric in ['SP_TR']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_AVS)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_SED)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_AVS,
                                          calc_SED)
            elif metric in ['SP_DP']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_AVS)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_DP)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_AVS,
                                          calc_DP)
            elif metric in ['TR_DP']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_SED)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_DP)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_SED,
                                          calc_DP)
            elif metric in ['DP_SP']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_DP)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_AVS)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_DP,
                                          calc_AVS)
            elif metric in ['DP_TR']:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_DP)
                max_epsilon2, idx2, epsilon2 = traj_max_dists(curr_traj, traj_time, calc_SED)
                compress_traj = calc_func(curr_traj, dim_set, traj_time, epsilon * alpha, epsilon2 * alpha, calc_DP,
                                          calc_SED)
            else:
                max_epsilon, idx, epsilon = traj_max_dists(curr_traj, traj_time, calc_func)
                compress_traj = traj_compression(curr_traj, dim_set, traj_time, calc_func, epsilon * alpha)
        except:
            print(
                f"\t\tIt was not possible to compress this trajectory {mmsis[id_mmsi]} of length {len(curr_traj['lat'])}.")

        compress_traj['DateTime'] = compress_traj['DateTime'].astype('datetime64[s]')
        new_dataset[mmsis[id_mmsi]] = compress_traj
        t1 = time.time() - t0
        if verbose:
            print(f"\tlength before: {len(curr_traj['lat'])}, length now: {len(compress_traj['lat'])}, reduction of {1 - len(compress_traj['lat'])/len(curr_traj['lat'])}")
        compression_rate = np.append(compression_rate, 1 - (len(compress_traj['lat']) / len(curr_traj['lat'])))
        processing_time = np.append(processing_time, t1)

    return new_dataset, compression_rate, processing_time

In [4]:
%%time

# Original Compression.
new_dataset = {}
ids = dataset.reset_index()['traj_id'].unique()
for id in ids:
    # getting one trajectory
    trajectory = dataset.reset_index()[dataset.reset_index()['traj_id'] == id]
    trajectory.set_index(['traj_id'])

    # converting trajectory to dict
    new_dataset[id] = {}
    for col in trajectory.columns:
        new_dataset[id][col] = np.array(trajectory[col])

new_dataset, compression_rate, processing_time = compression(new_dataset, verbose=True)
new_dataset = Conversions.dict_to_pandas(new_dataset)

Dim set is:  dict_keys(['traj_id', 'DateTime', 'trajectory', 'lat', 'lon', 'sog', 'cog', 'vessel_type', 'flag'])
Compressing with TR and factor 1
	Compressing 0 of 55
	length before: 157, length now: 5, reduction of 0.9681528662420382
	Compressing 1 of 55
	length before: 69, length now: 4, reduction of 0.9420289855072463
	Compressing 2 of 55
	length before: 83, length now: 4, reduction of 0.9518072289156626
	Compressing 3 of 55
	length before: 740, length now: 12, reduction of 0.9837837837837837
	Compressing 4 of 55
	length before: 224, length now: 4, reduction of 0.9821428571428571
	Compressing 5 of 55
	length before: 489, length now: 4, reduction of 0.9918200408997955
	Compressing 6 of 55
	length before: 1927, length now: 6, reduction of 0.9968863518422418
	Compressing 7 of 55
	length before: 132, length now: 10, reduction of 0.9242424242424242
	Compressing 8 of 55
	length before: 1190, length now: 25, reduction of 0.9789915966386554
	Compressing 9 of 55
	length before: 6, length now

In [5]:
new_dataset

,traj_id,DateTime,trajectory,lat,lon,sog,cog,vessel_type,flag
0,316002721,2020-04-06 05:42:48,0.0,37.6018,-122.6299,9.2,316.2,30.0,Canada
1,316002721,2020-04-06 08:00:17,0.0,37.8587,-122.8962,7.8,313.6,30.0,Canada
2,316002721,2020-04-06 08:01:48,0.0,37.8614,-122.8991,8.5,319.6,30.0,Canada
3,316002721,2020-04-07 19:29:51,0.0,37.8201,-122.8488,8.6,144.5,30.0,Canada
4,316002721,2020-04-07 21:15:50,0.0,37.6012,-122.6846,8.8,145.0,30.0,Canada
...,...,...,...,...,...,...,...,...,...
9,368141480,2020-05-31 00:10:31,54.0,37.7029,-122.8368,9.8,314.0,30.0,United States of America
10,368141480,2020-05-31 00:11:33,54.0,37.7050,-122.8392,10.4,323.5,30.0,United States of America
11,368141480,2020-05-31 00:13:51,54.0,37.7110,-122.8427,10.1,343.9,30.0,United States of America
12,368141480,2020-05-31 00:15:03,54.0,37.7139,-122.8447,10.1,326.3,30.0,United States of America


In [6]:
# PTRAIL Compression.
new_dataset1, compression_rate1 = Compression.compress_trajectories(dataset, verbose=True)

Compressing with TR and factor 1.
	Compressing 1 of 55
	length before: 157, length now: 5, reduction of: 0.9681528662420382
	Compressing 2 of 55
	length before: 69, length now: 4, reduction of: 0.9420289855072463
	Compressing 3 of 55
	length before: 83, length now: 4, reduction of: 0.9518072289156626
	Compressing 4 of 55
	length before: 740, length now: 12, reduction of: 0.9837837837837837
	Compressing 5 of 55
	length before: 224, length now: 4, reduction of: 0.9821428571428571
	Compressing 6 of 55
	length before: 489, length now: 4, reduction of: 0.9918200408997955
	Compressing 7 of 55
	length before: 1927, length now: 6, reduction of: 0.9968863518422418
	Compressing 8 of 55
	length before: 132, length now: 10, reduction of: 0.9242424242424242
	Compressing 9 of 55
	length before: 1190, length now: 25, reduction of: 0.9789915966386554
	Compressing 10 of 55
	length before: 6, length now: 4, reduction of: 0.33333333333333337
	Compressing 11 of 55
	length before: 154, length now: 4, reduc

In [7]:
new_dataset1

trajectory      lat       lon   sog    cog  \
traj_id   DateTime                                                         
316002721 2020-04-06 05:42:48        0.0  37.6018 -122.6299   9.2  316.2   
          2020-04-06 08:00:17        0.0  37.8587 -122.8962   7.8  313.6   
          2020-04-06 08:01:48        0.0  37.8614 -122.8991   8.5  319.6   
          2020-04-07 19:29:51        0.0  37.8201 -122.8488   8.6  144.5   
          2020-04-07 21:15:50        0.0  37.6012 -122.6846   8.8  145.0   
...                                  ...      ...       ...   ...    ...   
368141480 2020-05-31 00:10:31       54.0  37.7029 -122.8368   9.8  314.0   
          2020-05-31 00:11:33       54.0  37.7050 -122.8392  10.4  323.5   
          2020-05-31 00:13:51       54.0  37.7110 -122.8427  10.1  343.9   
          2020-05-31 00:15:03       54.0  37.7139 -122.8447  10.1  326.3   
          2020-05-31 00:43:02       54.0  37.7813 -122.8983  10.4  324.3   

                              vessel_type                      flag  
traj_id   DateTime                                                   
316002721 2020-04-06 05:42:48        30.0                    Canada  
          2020-04-06 08:00:17        30.0                    Canada  
          2020-04-06 08:01:48        30.0                    Canada  
          2020-04-07 19:29:51        30.0                    Canada  
          2020-04-07 21:15:50        30.0                    Canada  
...                                   ...                       ...  
368141480 2020-05-31 00:10:31        30.0  United States of America  
          2020-05-31 00:11:33        30.0  United States of America  
          2020-05-31 00:13:51        30.0  United States of America  
          2020-05-31 00:15:03        30.0  United States of America  
          2020-05-31 00:43:02        30.0  United States of America  

[621 rows x 7 columns]